In [1]:
#Bootstrap 
import sys 
import os
import sklearn
assert sklearn.__version__ >= '0.20'
import numpy as np
import pandas as pd

In [2]:
from os import listdir
from os.path import isfile, join


def read_data_csv(path=None,file_names=None):
    if (os.path.isfile(path)):
            data = pd.read_csv(path, header=0)
            print("Single File: "+ path)
        #directory   
    else:
        print("Multiple Files: " + path)
        li = []
        for file in file_names:
            file = join(path, file)
            df = pd.read_csv(file, index_col=None, header=0)
            li.append(df)
        data = pd.concat(li, axis=0)
    return data


        

In [3]:
def read_directory(path=None):
    path = 'new_england_datasets/'
    files = combine_files = [f for f in listdir(path) if isfile(join(path, f))]
    return files


In [4]:
read_directory()

['addison_county_vt_redfin_2020-05-05-07-46-31.csv',
 'androscoggin_county_me_redfin_2020-05-05-07-20-49.csv',
 'aroostook_county_me_redfin_2020-05-05-07-21-21.csv',
 'barnstable_county_ma_redfin_2020-05-05-06-42-51.csv',
 'belknap_county_nh_redfin_2020-05-05-07-35-24.csv',
 'bennington_county_vt_redfin_2020-05-05-07-47-23.csv',
 'bristol_county_ma_redfin_2020-05-05-06-43-26.csv',
 'bristol_county_ri_redfin_2020-05-05-07-42-12.csv',
 'caledonia_county_vt_redfin_2020-05-05-07-47-46.csv',
 'carroll_county_nh_redfin_2020-05-05-07-35-42.csv',
 'cheshire_county_nh_redfin_2020-05-05-07-35-57.csv',
 'chittenden_county_vt_redfin_2020-05-05-07-48-20.csv',
 'coos_county_nh_redfin_2020-05-05-07-36-12.csv',
 'cumberland_county_me_redfin_2020-05-05-07-21-52.csv',
 'dukes_county_ma_redfin_2020-05-05-06-44-17.csv',
 'essex_county_ma_redfin_2020-05-05-06-44-42.csv',
 'essex_county_vt_redfin_2020-05-05-07-48-39.csv',
 'fairfield_county_ct_redfin_2020-03-19-10-18-28.csv',
 'fairfield_county_ct_redfin_20

In [5]:
path = 'new_england_datasets/'
combine_files = [f for f in listdir(path) if isfile(join(path, f))]

data = read_data_csv(path=path, file_names=combine_files)


#keeping city
data  = data.drop( ["SOLD DATE", "ADDRESS","SALE TYPE", "ZIP OR POSTAL CODE",
            "URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)",
            "STATUS", "SOURCE", "FAVORITE","NEXT OPEN HOUSE START TIME", "NEXT OPEN HOUSE END TIME", 
          "INTERESTED", "LOCATION", "HOA/MONTH", "STATE OR PROVINCE", "$/SQUARE FEET",
                ], axis=1)

#maybe reset the index since the combining of files 
data = data.set_index(data['MLS#'])
#then drop the mls bc isn't needed
data = data.drop(columns=['MLS#'])


#data.count()

Multiple Files: new_england_datasets/


In [6]:
# Figure a better way to do this 

# data[data['PROPERTY TYPE'] == 'Vacant Land']


#get the indexes of the property types wished to be dropped
drop_vacant = data[data['PROPERTY TYPE'] == 'Vacant Land'].index
drop_parking = data[data['PROPERTY TYPE'] == 'Parking'].index
drop_city_bath = data[data['CITY'] == 'Bath'].index 
drop_city_6 = data[data['CITY'] == 6.0].index
drop_t3_city = data[data['CITY'] == 'T3 R12 WELS'].index


data = data.drop(list(drop_vacant))
data = data.drop(list(drop_parking))
data = data.drop(list(drop_city_bath))
data = data.drop(list(drop_city_6))
data = data.drop(list(drop_t3_city))
#could also set all nulls in vacant land to 0, but take the median value prior 

# data[data['PROPERTY TYPE'] == 'Vacant Land'].count(), data[data['PROPERTY TYPE'] == 'Parking'].count()
data.count()

PROPERTY TYPE     8522
CITY              8521
PRICE             8522
BEDS              8516
BATHS             8474
SQUARE FEET       8509
LOT SIZE          6887
YEAR BUILT        8484
DAYS ON MARKET    8514
LATITUDE          8522
LONGITUDE         8522
dtype: int64

In [7]:
list(data.groupby('CITY'))


[('Abbot',
                       PROPERTY TYPE   CITY   PRICE  BEDS  BATHS  SQUARE FEET  \
  MLS#                                                                          
  1450950  Single Family Residential  Abbot  205900   8.0    3.5       4200.0   
  
           LOT SIZE  YEAR BUILT  DAYS ON MARKET   LATITUDE  LONGITUDE  
  MLS#                                                                 
  1450950  392040.0      2000.0             4.0  45.172746  -69.51492  ),
 ('Abington',
                        PROPERTY TYPE      CITY   PRICE  BEDS  BATHS  \
  MLS#                                                                 
  72646358  Single Family Residential  Abington  459900   3.0    1.0   
  72650422    Multi-Family (2-4 Unit)  Abington  399000   7.0    3.0   
  72649669  Single Family Residential  Abington  429000   4.0    1.5   
  72649452  Single Family Residential  Abington  459000   3.0    1.5   
  72646358  Single Family Residential  Abington  459900   3.0    1.0   
  
    

In [8]:
#remove categories that are less than 30 from property history, nvm removes too many 
# data = data.groupby('PROPERTY TYPE').filter(lambda x: len(x) > 30)

#replace null values of 'days on market' with median
data = data.fillna(data["DAYS ON MARKET"].median())

#fill the lot size with median as well
data = data.fillna(data['LOT SIZE'].median())
data = data.fillna(data['SQUARE FEET'].median())

#median numbers to fill later, based on columns
median_values = data.median().values

#need to drop rows that have one entry for city and property type
data = data.groupby(['CITY', 'PROPERTY TYPE']).filter(lambda x: len(x)>4)

data.count()

PROPERTY TYPE     6734
CITY              6734
PRICE             6734
BEDS              6734
BATHS             6734
SQUARE FEET       6734
LOT SIZE          6734
YEAR BUILT        6734
DAYS ON MARKET    6734
LATITUDE          6734
LONGITUDE         6734
dtype: int64

In [9]:
#check for null values
data[data.isnull().any(axis=1)].head()

Empty DataFrame
Columns: [PROPERTY TYPE, CITY, PRICE, BEDS, BATHS, SQUARE FEET, LOT SIZE, YEAR BUILT, DAYS ON MARKET, LATITUDE, LONGITUDE]
Index: []

In [10]:
#it is possible to stratify here 
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data,
                                       test_size=0.2, 
                                       random_state=42,
                                       stratify=data["PROPERTY TYPE"] + data['CITY']
                                      )
#split for validation set
x_to_split = train_set.drop(["PRICE",
                ], axis=1)
y_to_split = train_set["PRICE"].copy()

#Split on Property History, otherwise the Cats: Townhouse, Parking maybe omitted from a set
#appears that there might not be enough data to split by property and city as well
X_train, X_valid, y_train, y_valid = train_test_split(x_to_split, y_to_split, 
                                                      stratify= x_to_split["PROPERTY TYPE"]
                                                      + x_to_split['CITY'],
                                                      random_state=42)

#same for the test set
X_test = test_set.drop(["PRICE"], axis=1)
y_test = test_set["PRICE"].copy()

In [11]:
#double check 
x_to_split.groupby(['PROPERTY TYPE', 'CITY']).filter(lambda x: len(x)> 4).count()

PROPERTY TYPE     5091
CITY              5091
BEDS              5091
BATHS             5091
SQUARE FEET       5091
LOT SIZE          5091
YEAR BUILT        5091
DAYS ON MARKET    5091
LATITUDE          5091
LONGITUDE         5091
dtype: int64

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer



#all columns other than property category
numerical_attributes = list(X_train.drop(["PROPERTY TYPE", 'CITY'], axis=1))
#Property and Cities
categorical_attributes = list(X_train[["PROPERTY TYPE", "CITY"]])





num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])



categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder(sparse=False)),
])

In [13]:
#another option is to prep the data into a simple pipeline, this example utilizes a seperate numerical and categorical
from sklearn.compose import ColumnTransformer



full_pipeline = ColumnTransformer([
    ("numerical_attributes", num_pipeline, numerical_attributes),
    ("categorical_attributes", categorical_pipeline, categorical_attributes),
 
])

x_train_prepared = full_pipeline.fit_transform(X_train)
x_valid_prepared = full_pipeline.fit_transform(X_valid)
x_test_prepared = full_pipeline.fit_transform(X_test)

In [14]:
#try training now
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(x_train_prepared, y_train)
#score is the r2
lin_reg.score(x_train_prepared, y_train)


0.6069940017690608

In [15]:
y_test

MLS#
170274140     359900
4803108       339900
72644774      329000
72648526      579000
72611479     2950000
170282220     250000
170290307     189900
170282007     110000
72649889      489900
170279996     132000
170265254     249900
4803717       675000
72644992     1995000
170276108     479000
72651463      305000
72645668      270000
170270870     847900
1451046       479000
170287215     229900
170277005     530000
170269535     132500
170290833     509000
4802033       455000
72649215      429900
170290563    4350000
170281389     211500
170279473     449000
1450532       579000
1251346       329900
170280632     204900
              ...   
72650670      579900
72648502      629000
170267905     299900
170292385     419900
170291043     669000
170281149     139900
170279559     159900
170292045     229900
170290429     349900
170290030     635000
170282337     219000
72572416     1295000
72649882      849900
170273910     200000
4803071       389900
170276322     339900
17027627

In [16]:
#lets see how it preformed 
# some_data = x_test.iloc[:5]

# some_labels = y_test.iloc[:5]


# some_data_prepd = full_pipeline.transform(some_data)
# # some_data_prepd.shape
# lin_reg.predict(some_data_prepd)

# print('Predictions: ', lin_reg.predict(some_data_prepd))
# print('actual:', list(some_labels))



In [17]:
#check the mean square error for performance of the linear regression
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(x_train_prepared)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

464554.85081277956

In [18]:
def compare_scores(model, x, y):
    print('Predictions: ', model.predict(x))
    print('actual:', list(y))


In [19]:
#using a decision tree model rather than the linear regression
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(x_train_prepared, y_train)
tree_reg.score(x_test_prepared, y_test)

0.3572961954325037

In [20]:
# compare_scores(tree_reg, some_data_prepd, some_labels)


In [21]:
#fine tune the model by splitting the training/test data into representative groups
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, x_train_prepared, y_train, scoring="neg_mean_squared_error", cv=10)
tree_revised_rmnse_scores = np.sqrt(-scores)

In [22]:
lin_scores = cross_val_score(lin_reg, x_train_prepared, y_train, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)


In [23]:
decision_tree_predictions = tree_reg.predict(x_train_prepared)
tree_mse = mean_squared_error(y_train, decision_tree_predictions)
tree_root_mse = np.sqrt(tree_mse)
tree_root_mse

884.6882317103199

In [24]:
#uses cross validation scores 
# E.G.cross_val_score(tree_reg, x_train_prepared, y_train, scoring="neg_mean_squared_error", cv=10)
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard_deviation: ", scores.std())
display_scores(tree_revised_rmnse_scores)

Scores:  [ 414570.23201529  975526.41072784  479882.48622255 1215255.52697581
  470951.8872144   513295.7683028   706236.21215734  365495.85075421
  739313.40113189  570977.01639817]
Mean:  645150.4791900303
Standard_deviation:  256648.81301017458


In [25]:
#the linear regression actually preforms better on this smaller training set
linear_regression_scores = cross_val_score(lin_reg, x_train_prepared, y_train, scoring='neg_mean_squared_error', cv=10)
linear_regression_rmse_scores = np.sqrt(-linear_regression_scores)
display_scores(linear_regression_rmse_scores)

Scores:  [3.62348811e+05 8.82252005e+05 3.85544408e+05 5.66586028e+05
 4.67862168e+05 4.67300926e+05 6.21077849e+05 7.56745158e+13
 4.87383634e+05 1.99381003e+06]
Mean:  7567452203374.341
Standard_deviation:  22702354532067.74


In [26]:
pd.Series(np.sqrt(-scores)).describe()

count    1.000000e+01
mean     6.451505e+05
std      2.705316e+05
min      3.654959e+05
25%      4.731845e+05
50%      5.421364e+05
75%      7.310441e+05
max      1.215256e+06
dtype: float64

In [27]:
#SVR model
from sklearn.svm import SVR 

#i should make a function to call each modules easily with the fit, etc. like below
svm_regression = SVR(kernel='linear')
svm_regression.fit(x_train_prepared, y_train)
housing_predictions = svm_regression.predict(x_train_prepared)
svm_mse = mean_squared_error(y_train, housing_predictions)
#unsure why sometimes it was run 
svm_rmse = np.sqrt(svm_mse)
svm_rmse

760837.8183390093

In [28]:
# compare_scores(svm_regression, some_data_prepd, some_labels)

In [29]:
#check multiple types of models at once 

from sklearn.model_selection import GridSearchCV


#the first attempt will be 12 different combinations (3 estimators x 4 features)
#the second attempt will do 2 estimators by 3 features so 6 combinations
param_grid = [
    #estimators are the number of 'trees', 'features' are the parameters
    {'n_estimators':[3,10,3], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features': [2,3,4]},
    
]

In [30]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(x_train_prepared, y_train)
forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                          verbose=2)
grid_search.fit(x_train_prepared, y_train)



Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] max_features=2, n_estimators=3 ..................................
[CV] ................... max_features=2, n_estimators=3, total=   0.1s
[CV] max_features=2, n_estimators=3 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ................... max_features=2, n_estimators=3, total=   0.1s
[CV] max_features=2, n_estimators=3 ..................................
[CV] ................... max_features=2, n_estimators=3, total=   0.1s
[CV] max_features=2, n_estimators=3 ..................................
[CV] ................... max_features=2, n_estimators=3, total=   0.1s
[CV] max_features=2, n_estimators=3 ..................................
[CV] ................... max_features=2, n_estimators=3, total=   0.1s
[CV] max_features=2, n_estimators=10 .................................
[CV] .................. max_features=2, n_estimators=10, total=   0.2s
[CV] max_features=2, n_estimators=10 .................................
[CV] .................. max_features=2, n_estimators=10, total=   0.3s
[CV] max_features=2, n_estimators=10 .................................
[CV] .................. max_features=2, n_estimators=10, total=   0.3s
[CV] max_features=2, n_estimators=10 .................................
[CV] .

[CV] .. bootstrap=False, max_features=2, n_estimators=3, total=   0.1s
[CV] bootstrap=False, max_features=2, n_estimators=3 .................
[CV] .. bootstrap=False, max_features=2, n_estimators=3, total=   0.1s
[CV] bootstrap=False, max_features=2, n_estimators=3 .................
[CV] .. bootstrap=False, max_features=2, n_estimators=3, total=   0.1s
[CV] bootstrap=False, max_features=2, n_estimators=3 .................
[CV] .. bootstrap=False, max_features=2, n_estimators=3, total=   0.1s
[CV] bootstrap=False, max_features=2, n_estimators=3 .................
[CV] .. bootstrap=False, max_features=2, n_estimators=3, total=   0.1s
[CV] bootstrap=False, max_features=2, n_estimators=10 ................
[CV] . bootstrap=False, max_features=2, n_estimators=10, total=   0.3s
[CV] bootstrap=False, max_features=2, n_estimators=10 ................
[CV] . bootstrap=False, max_features=2, n_estimators=10, total=   0.2s
[CV] bootstrap=False, max_features=2, n_estimators=10 ................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:   13.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [2, 4, 6, 8],
  

In [31]:
#show the best estimator
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [32]:
#look at the results of the grid search 
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(np.sqrt(-mean_score), params)

589294.8793456065 {'max_features': 2, 'n_estimators': 3}
509387.97397744306 {'max_features': 2, 'n_estimators': 10}
589294.8793456065 {'max_features': 2, 'n_estimators': 3}
577208.3724404275 {'max_features': 4, 'n_estimators': 3}
513481.35434717 {'max_features': 4, 'n_estimators': 10}
577208.3724404275 {'max_features': 4, 'n_estimators': 3}
560755.1512882202 {'max_features': 6, 'n_estimators': 3}
490471.7521580063 {'max_features': 6, 'n_estimators': 10}
560755.1512882202 {'max_features': 6, 'n_estimators': 3}
570465.6500258466 {'max_features': 8, 'n_estimators': 3}
521909.01773393125 {'max_features': 8, 'n_estimators': 10}
570465.6500258466 {'max_features': 8, 'n_estimators': 3}
594180.4098382631 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
510560.13571186044 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
542162.523021356 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
494176.6058069655 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}
57

In [33]:
#when looking for more combinations try randomcvsearch
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_regression = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(forest_regression, param_distributions=param_distribs,
                                  n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, verbose=2)
random_search.fit(x_train_prepared, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] max_features=7, n_estimators=180 ................................
[CV] ................. max_features=7, n_estimators=180, total=   1.5s
[CV] max_features=7, n_estimators=180 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV] ................. max_features=7, n_estimators=180, total=   1.4s
[CV] max_features=7, n_estimators=180 ................................
[CV] ................. max_features=7, n_estimators=180, total=   1.6s
[CV] max_features=7, n_estimators=180 ................................
[CV] ................. max_features=7, n_estimators=180, total=   2.7s
[CV] max_features=7, n_estimators=180 ................................
[CV] ................. max_features=7, n_estimators=180, total=   3.0s
[CV] max_features=5, n_estimators=15 .................................
[CV] .................. max_features=5, n_estimators=15, total=   0.2s
[CV] max_features=5, n_estimators=15 .................................
[CV] .................. max_features=5, n_estimators=15, total=   0.3s
[CV] max_features=5, n_estimators=15 .................................
[CV] .................. max_features=5, n_estimators=15, total=   0.2s
[CV] max_features=5, n_estimators=15 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.0min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [34]:
feature_importances = grid_search.best_estimator_.feature_importances_
best_forest_model = grid_search.best_estimator_

In [35]:
# displaying the importance scores next to their columns
category_encoder = full_pipeline.named_transformers_["categorical_attributes"]["one_hot"]

cat_one_hot_attributes = list(category_encoder.categories_[0])

attributes = numerical_attributes + cat_one_hot_attributes
sorted(zip(feature_importances, attributes), reverse=True)

[(0.18682477248315907, 'SQUARE FEET'),
 (0.1510383466323979, 'BATHS'),
 (0.09969818863336356, 'LONGITUDE'),
 (0.07097223152272919, 'YEAR BUILT'),
 (0.06220778184655689, 'LATITUDE'),
 (0.06060816850381472, 'LOT SIZE'),
 (0.05454031603791756, 'BEDS'),
 (0.054124389638804386, 'DAYS ON MARKET'),
 (0.007236855100525376, 'Single Family Residential'),
 (0.0029683758591162643, 'Condo/Co-op'),
 (0.0017678244039370506, 'Multi-Family (2-4 Unit)'),
 (0.00029776910920655556, 'Townhouse')]

In [36]:
#I should probably remove the $/square feet, as there probably is fairly high correlation between the two
#also latitude seems like an odd predictor of price, as all data is within the town..
category_encoder.categories_


[array(['Condo/Co-op', 'Multi-Family (2-4 Unit)',
        'Single Family Residential', 'Townhouse'], dtype=object),
 array(['Acton', 'Acushnet', 'Amesbury', 'Amherst', 'Andover', 'Ansonia',
        'Arlington', 'Ashburnham', 'Ashford', 'Attleboro', 'Auburn',
        'Augusta', 'Avon', 'Bangor', 'Barkhamsted', 'Barnstable',
        'Barnstead', 'Barrington', 'Beacon Falls', 'Bedford', 'Belgrade',
        'Bennington', 'Berlin', 'Bethel', 'Bethlehem', 'Beverly',
        'Biddeford', 'Billerica', 'Bloomfield', 'Bolton', 'Boston',
        'Bourne', 'Boxford', 'Braintree', 'Branford', 'Brattleboro',
        'Brewer', 'Brewster', 'Bridgeport', 'Bridgewater', 'Bridgton',
        'Bristol', 'Brockton', 'Brookfield', 'Brookline', 'Brooklyn',
        'Brunswick', 'Burlington', 'Cambridge', 'Canaan', 'Canterbury',
        'Canton', 'Cape Elizabeth', 'Caribou', 'Chaplin', 'Charlestown',
        'Charlton', 'Chatham', 'Chelmsford', 'Chelsea', 'Cheshire',
        'Chester', 'Clinton', 'Cohasset', 'C

In [37]:
final_model = grid_search.best_estimator_ 
final_model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [38]:
final_predictions = final_model.predict(x_test_prepared)
#dataset still not large enough!
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [39]:
#w/out latitude and longitude model was massively off
final_rmse

590132.4367075351

In [40]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [41]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

array([243084.96389075, 798387.30328017])

In [42]:
#putting the data into a neural network
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [43]:
#should correct the issue...
y_train = (y_train.astype(np.float32))
y_test = (y_test.astype(np.float32))
y_valid = (y_valid.astype(np.float32))


ValueError: Error when checking input: expected dense_93_input to have shape (16,) but got array with shape (15,)

In [44]:
#okay the valid shape is 15 for some reason
x_valid_prepared.shape

(1347, 374)

In [45]:
tf.random.set_seed(42)
np.random.seed(42)
len(data.columns)

input_shape = x_train_prepared.shape[1:]
#simple nn
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [46]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["accuracy", "mse"])

In [47]:
#poor model, no accuracy
history = model.fit(x_train_prepared, y_train, epochs=10,
             validation_data=(x_valid_prepared, y_valid))

Train on 4040 samples, validate on 1347 samples
Epoch 1/10
4040/4040 [==============================] - 1s 309us/sample - loss: 550974.3669 - accuracy: 0.0000e+00 - mse: 852702593024.0000 - val_loss: 529687.9929 - val_accuracy: 0.0000e+00 - val_mse: 662013083648.0000
Epoch 2/10
4040/4040 [==============================] - 0s 99us/sample - loss: 550967.2754 - accuracy: 0.0000e+00 - mse: 852694859776.0000 - val_loss: 529678.1741 - val_accuracy: 0.0000e+00 - val_mse: 662002466816.0000
Epoch 3/10
4040/4040 [==============================] - 0s 105us/sample - loss: 550955.1613 - accuracy: 0.0000e+00 - mse: 852681424896.0000 - val_loss: 529663.7090 - val_accuracy: 0.0000e+00 - val_mse: 661987328000.0000
Epoch 4/10
4040/4040 [==============================] - 0s 99us/sample - loss: 550938.7031 - accuracy: 0.0000e+00 - mse: 852663533568.0000 - val_loss: 529645.1374 - val_accuracy: 0.0000e+00 - val_mse: 661967732736.0000
Epoch 5/10
4040/4040 [==============================] - 0s 90us/sample - l

# Trying more complex models

In [48]:
input_shape

(374,)

In [49]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [104]:
base_opt = keras.optimizers

In [105]:
parameters_distributions = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons":np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 9e-2),
    'optimizer':[ base_opt.SGD(), base_opt.RMSprop() ]
}

In [106]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[374], optimizer=None):
    model = keras.models.Sequential()
    #input layer
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    #hidden layers
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='elu'))
    #output layer
    model.add(keras.layers.Dense(1))
    #optimizer 
    optimizer = keras.optimizers.RMSprop()
    #compile
    model.compile(loss='mse', optimizer=optimizer)
    return model


keras_regressor = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [107]:
#n_iterations = number of parameter settings that are sampled
#cv = cross folds of the data
rand_search_cv = RandomizedSearchCV(keras_regressor, parameters_distributions, n_iter=10, 
                                   cv=3, verbose=2)

In [108]:
rand_search_cv.fit(x_train_prepared, y_train, epochs=100, 
                   validation_data=(x_valid_prepared, y_valid), 
                   callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.0001373604440128277, n_hidden=3, n_neurons=80, optimizer=<tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x7f21770f3c18> 
Train on 2693 samples, validate on 1347 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


2693/2693 [==============================] - 1s 206us/sample - loss: 868622583384.4070 - val_loss: 661836265719.8278
Epoch 2/100
2693/2693 [==============================] - 0s 85us/sample - loss: 868121223669.9235 - val_loss: 660938892754.0074
Epoch 3/100
2693/2693 [==============================] - 0s 104us/sample - loss: 866365829847.0286 - val_loss: 658559006311.0082
Epoch 4/100
2693/2693 [==============================] - 0s 102us/sample - loss: 862446987549.1838 - val_loss: 653325911378.2925
Epoch 5/100
2693/2693 [==============================] - 0s 95us/sample - loss: 854600629158.6423 - val_loss: 644411952838.7943
Epoch 6/100
2693/2693 [==============================] - 0s 94us/sample - loss: 842608669653.0322 - val_loss: 630414755324.5791
Epoch 7/100
2693/2693 [==============================] - 0s 102us/sample - loss: 824666586220.3698 - val_loss: 610804023867.6763
Epoch 8/100
2693/2693 [==============================] - 0s 94us/sample - loss: 798069944031.0137 - val_loss: 58

2693/2693 [==============================] - 0s 73us/sample - loss: 238623204370.2518 - val_loss: 164693054857.0275
Epoch 65/100
2693/2693 [==============================] - 0s 70us/sample - loss: 238083948635.2588 - val_loss: 164115312497.0809
Epoch 66/100
2693/2693 [==============================] - 0s 88us/sample - loss: 237255931393.7111 - val_loss: 165078601998.6340
Epoch 67/100
2693/2693 [==============================] - 0s 80us/sample - loss: 235464155709.4096 - val_loss: 163643975350.0698
Epoch 68/100
2693/2693 [==============================] - 0s 85us/sample - loss: 235285332024.2763 - val_loss: 164148774271.9050
Epoch 69/100
2693/2693 [==============================] - 0s 83us/sample - loss: 234577930391.3376 - val_loss: 163695433267.3140
Epoch 70/100
2693/2693 [==============================] - 0s 71us/sample - loss: 233391637015.3850 - val_loss: 163774352301.1373
Epoch 71/100
2693/2693 [==============================] - 0s 86us/sample - loss: 232681447749.4898 - val_loss:

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.7s remaining:    0.0s


...

2694/2694 [==============================] - 0s 67us/sample - loss: 888075588079.6555 - val_loss: 660969847931.9139
Epoch 94/100
2694/2694 [==============================] - 0s 78us/sample - loss: 888050725089.0215 - val_loss: 660947838150.4143
Epoch 95/100
2694/2694 [==============================] - 0s 71us/sample - loss: 888026514004.0029 - val_loss: 660925423086.8953
Epoch 96/100
2694/2694 [==============================] - 0s 70us/sample - loss: 888003375502.3489 - val_loss: 660903964413.5293
Epoch 97/100
2694/2694 [==============================] - 0s 70us/sample - loss: 887978497663.3348 - val_loss: 660881824289.8292
Epoch 98/100
2694/2694 [==============================] - 0s 79us/sample - loss: 887953438748.8879 - val_loss: 660858978912.1663
Epoch 99/100
2694/2694 [==============================] - 0s 79us/sample - loss: 887927387702.3549 - val_loss: 660835328103.3883
Epoch 100/100
1346/1346 [==============================] - 0s 30us/sample - loss: 778444661661.0996
[CV]  lear

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.1min finished


4040/4040 [==============================] - 1s 161us/sample - loss: 852477233342.6060 - val_loss: 661245291446.2599
Epoch 2/100
4040/4040 [==============================] - 0s 97us/sample - loss: 849947928685.4971 - val_loss: 656362784544.4989
Epoch 3/100
4040/4040 [==============================] - 0s 96us/sample - loss: 840037588262.0198 - val_loss: 642230941201.8649
Epoch 4/100
4040/4040 [==============================] - 0s 83us/sample - loss: 818199447742.6060 - val_loss: 613333427045.6779
Epoch 5/100
4040/4040 [==============================] - 0s 85us/sample - loss: 776268840895.1129 - val_loss: 563081566820.7275
Epoch 6/100
4040/4040 [==============================] - 0s 87us/sample - loss: 712117396948.4039 - val_loss: 493840971555.5397
Epoch 7/100
4040/4040 [==============================] - 0s 89us/sample - loss: 634178712387.4218 - val_loss: 412706141151.3110
Epoch 8/100
4040/4040 [==============================] - 0s 94us/sample - loss: 536668059684.4990 - val_loss: 32663

4040/4040 [==============================] - 0s 83us/sample - loss: 230360641576.5544 - val_loss: 155184830866.9102
Epoch 65/100
4040/4040 [==============================] - 0s 84us/sample - loss: 228953657072.2851 - val_loss: 153138736306.6488
Epoch 66/100
4040/4040 [==============================] - 0s 76us/sample - loss: 229205308324.7525 - val_loss: 152791060495.2042
Epoch 67/100
4040/4040 [==============================] - 0s 85us/sample - loss: 228624213719.9525 - val_loss: 152090357218.7320
Epoch 68/100
4040/4040 [==============================] - 0s 83us/sample - loss: 227932060380.0079 - val_loss: 151788183126.2836
Epoch 69/100
4040/4040 [==============================] - 0s 102us/sample - loss: 227274140495.5881 - val_loss: 152437211002.2034
Epoch 70/100
4040/4040 [==============================] - 0s 79us/sample - loss: 225836538233.1564 - val_loss: 152626400683.9970
Epoch 71/100
4040/4040 [==============================] - 0s 101us/sample - loss: 225641903716.3723 - val_los

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f217707eac8>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f21770f34e0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,...
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'optimizer': [<tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x7f21770f3c18>,
                                                      <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x7f21770f3208>]},
                   pre_dispatch='2*n_jobs', random_sta

In [54]:
rand_search_cv.best_params_

{'learning_rate': 0.0012778572479005168, 'n_hidden': 0, 'n_neurons': 15}

In [109]:
#the model still sucks...
best_model = rand_search_cv.best_estimator_.model
best_model.evaluate(x_test_prepared, y_test)

1347/1347 [==============================] - 0s 35us/sample - loss: 373038185193.3838


373038185193.3838

best model still performs quite poorly, need to get more data

In [110]:
rand_search_cv.best_score_, rand_search_cv.best_estimator_.score

(-277235392021.8297,
 <bound method KerasRegressor.score of <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f2177598630>>)

In [111]:
model = rand_search_cv.best_estimator_.model

In [112]:
attributes = numerical_attributes + cat_one_hot_attributes
sorted(zip(feature_importances, attributes), reverse=True)

[(0.18682477248315907, 'SQUARE FEET'),
 (0.1510383466323979, 'BATHS'),
 (0.09969818863336356, 'LONGITUDE'),
 (0.07097223152272919, 'YEAR BUILT'),
 (0.06220778184655689, 'LATITUDE'),
 (0.06060816850381472, 'LOT SIZE'),
 (0.05454031603791756, 'BEDS'),
 (0.054124389638804386, 'DAYS ON MARKET'),
 (0.007236855100525376, 'Single Family Residential'),
 (0.0029683758591162643, 'Condo/Co-op'),
 (0.0017678244039370506, 'Multi-Family (2-4 Unit)'),
 (0.00029776910920655556, 'Townhouse')]

In [113]:
model.evaluate(x_test_prepared, y_test)

1347/1347 [==============================] - 0s 53us/sample - loss: 373038185193.3838


373038185193.3838

In [60]:
y_test

MLS#
170274140     359900.0
4803108       339900.0
72644774      329000.0
72648526      579000.0
72611479     2950000.0
170282220     250000.0
170290307     189900.0
170282007     110000.0
72649889      489900.0
170279996     132000.0
170265254     249900.0
4803717       675000.0
72644992     1995000.0
170276108     479000.0
72651463      305000.0
72645668      270000.0
170270870     847900.0
1451046       479000.0
170287215     229900.0
170277005     530000.0
170269535     132500.0
170290833     509000.0
4802033       455000.0
72649215      429900.0
170290563    4350000.0
170281389     211500.0
170279473     449000.0
1450532       579000.0
1251346       329900.0
170280632     204900.0
               ...    
72650670      579900.0
72648502      629000.0
170267905     299900.0
170292385     419900.0
170291043     669000.0
170281149     139900.0
170279559     159900.0
170292045     229900.0
170290429     349900.0
170290030     635000.0
170282337     219000.0
72572416     1295000.0
726498

So the model still performs awful, could be the dataset size, I could also try and leave out some of houses that are in underpopulated areas as they might skew the results. e.g: vermont and nh counties had <20 in them.

In [61]:
len(X_train), len(X_valid), len(X_test)

(4040, 1347, 1347)

need to find a way to test multiple parameters, might also need more data too